## Лабораторная работа №5

Выполнил: Сайфуллин Динислам

Вариант 2

## Задание 1
В данной лабораторной работе рассматривается задача построения и оценки линейной регрессионной модели для анализа зависимости массы мобильного телефона от некоторых его характеристик: высоты экрана (sc_h), ширины экрана (sc_w) и мощности батареи (battery_power). Целью работы является:

1. Оценить параметры линейной модели  
   $$
   \text{mobile\_wt}_i = \beta_0 + \beta_1\,\mathrm{sc\_h}_i + \beta_2\,\mathrm{sc\_w}_i + \beta_3\,\mathrm{battery\_power}_i + \varepsilon_i,
   $$

2. Проверить статистическую значимость оценённых коэффициентов с помощью t-тестов (двусторонних и односторонних) и построить 95% доверительные интервалы для каждого параметра.

3. Оценить качество модели через коэффициент детерминации $R^2$ и остаточную дисперсию $\hat\sigma^2$.

4. Выполнить F-тест для проверки совместной гипотезы о незначимости двух предикторов ($\beta_{\text{sc\_w}} = \beta_{\text{battery\_power}} = 0$).

5. Интерпретировать полученные результаты и сделать выводы о влиянии параметров экрана и батареи на массу устройства.

В качестве исходных данных используется файл `mobile_phones.csv`, содержащий наблюдения по трём выбранным признакам и значению массы каждого телефона.

In [4]:
import pandas as pd
import numpy as np
from scipy.stats import t, f
from numpy.linalg import inv

path = 'mobile_phones.csv'
X_cols = ['sc_h', 'sc_w', 'battery_power']
y_col = 'mobile_wt'
alpha = 0.05

df = pd.read_csv(path)[[y_col] + X_cols].dropna()
n = df.shape[0]
p = len(X_cols) + 1

X = np.column_stack([np.ones(n), df[X_cols].values])
y = df[y_col].values.reshape(-1,1)

XtX = X.T @ X
beta = inv(XtX) @ (X.T @ y)
resid = y - X @ beta

RSS = (resid.T @ resid).item()
sigma2 = RSS / (n - p)
TSS = ((y - y.mean())**2).sum().item()
R2 = 1 - RSS / TSS

covb = sigma2 * inv(XtX)
se = np.sqrt(np.diag(covb)).reshape(-1,1)
df_res = n - p
t_crit = t.ppf(1 - alpha/2, df_res)

ci_lower = beta - t_crit*se
ci_upper = beta + t_crit*se

t_stats = beta.flatten() / se.flatten()
p_vals = 2*(1 - t.cdf(np.abs(t_stats), df_res))

t_crit_1s = t.ppf(1 - alpha, df_res)
one_sided = {
    'sc_h': {'t': t_stats[1], 'crit': t_crit_1s},
    'sc_w': {'t': t_stats[2], 'crit': t_crit_1s},
}

test_idx = [2, 3]
keep = [i for i in range(p) if i not in test_idx]
X_red = X[:, keep]
beta_r = inv(X_red.T @ X_red) @ (X_red.T @ y)
resid_r = y - X_red @ beta_r
RSS_r = (resid_r.T @ resid_r).item()  # и здесь

F_stat = ((RSS_r - RSS)/len(test_idx)) / (RSS/(n - p))
F_crit = f.ppf(1 - alpha, len(test_idx), n - p)
p_valF = 1 - f.cdf(F_stat, len(test_idx), n - p)

names = ['Intercept'] + X_cols
for i, name in enumerate(names):
    print(f"{name:>13}  β={beta[i,0]:7.3f}  SE={se[i,0]:6.3f}  "
          f"t={t_stats[i]:6.3f}  p(two-sided)={p_vals[i]:6.4f}  "
          f"CI=[{ci_lower[i,0]:6.3f}, {ci_upper[i,0]:6.3f}]")
print(f"\nR² = {R2:.4f}, σ²̂ = {sigma2:.4f}")

print("\nТ-ТЕСТЫ")
for var, v in one_sided.items():
    result = "Отклоняем H0" if v['t'] > v['crit'] else "Не отклоняем H0"
    print(f"{var:>6}  t={v['t']:.3f}  crit={v['crit']:.3f} -> {result}")

print("\nF-ТЕСТ СОВМЕСТНОЙ ГИПОТЕЗЫ β_sc_w=β_battery=0")
print(f"F={F_stat:.3f}, crit={F_crit:.3f}, p={p_valF:.4f}")
print("->", "Отклоняем H0" if F_stat>F_crit else "Не отклоняем H0")


    Intercept  β=143.641  SE= 3.361  t=42.743  p(two-sided)=0.0000  CI=[137.050, 150.231]
         sc_h  β= -0.264  SE= 0.218  t=-1.209  p(two-sided)=0.2268  CI=[-0.691,  0.164]
         sc_w  β= -0.040  SE= 0.211  t=-0.188  p(two-sided)=0.8512  CI=[-0.453,  0.374]
battery_power  β=  0.000  SE= 0.002  t= 0.036  p(two-sided)=0.9715  CI=[-0.003,  0.004]

R² = 0.0012, σ²̂ = 1253.5576

Т-ТЕСТЫ
  sc_h  t=-1.209  crit=1.646 -> Не отклоняем H0
  sc_w  t=-0.188  crit=1.646 -> Не отклоняем H0

F-ТЕСТ СОВМЕСТНОЙ ГИПОТЕЗЫ β_sc_w=β_battery=0
F=0.018, crit=3.000, p=0.9819
-> Не отклоняем H0


## Результаты

Ниже приведены основные выводы, непосредственно основанные на выводе программы:

1. **Оценки коэффициентов и их статистики**  
   - **Intercept**: β = 143.641, SE = 3.361, t = 42.743, p < 0.0001, 95% ДИ [137.050, 150.231]  
     > Вывод программы говорит, что свободный член модели статистически значим (очень маленькое p-значение).  
   - **sc_h** (высота экрана): β = –0.264, SE = 0.218, t = –1.209, p = 0.2268, 95% ДИ [–0.691, 0.164]  
     > Вывод программы показывает, что этот коэффициент незначим (p > 0.05 и доверительный интервал содержит ноль).  
   - **sc_w** (ширина экрана): β = –0.040, SE = 0.211, t = –0.188, p = 0.8512, 95% ДИ [–0.453, 0.374]  
     > Программа констатирует полное отсутствие статистически значимого эффекта.  
   - **battery_power**: β ≈ 0.000, SE = 0.002, t = 0.036, p = 0.9715, 95% ДИ [–0.003, 0.004]  
     > Коэффициент практически равен нулю и незначим.

2. **Качество модели**  
   - Программа вычислила $R^2 = 0.0012$.  
   - Оценка остаточной дисперсии: $\hat\sigma^2 = 1253.5576$.  
   > Это подтверждает, что модель объясняет лишь 0.12% вариации массы — почти никакой.

3. **Односторонние t-тесты (H₁: β > 0)**  
   - Для **sc_h** t = –1.209, критическое значение = 1.646 → Не отклоняем H₀.  
   - Для **sc_w** t = –0.188, критическое значение = 1.646 → Не отклоняем H₀.  
   > Программа выводит, что ни один из тестов не поддерживает альтернативную гипотезу, что коэффициенты положительные.

4. **F-тест совместной гипотезы β_sc_w = β_battery_power = 0**  
   - F = 0.018, критическое значение = 3.000, p = 0.9819 → Не отклоняем H₀.  
   > Согласно выводу программы, вместе эти два признака не вносят значимого вклада.

## Вывод

Вывод программы ясно демонстрирует:

- Статистическая значимость есть только у свободного члена (Intercept).  
- Ни высота экрана, ни ширина экрана, ни мощность батареи не являются значимыми предикторами массы в рамках данной модели.  
- Модель практически не объясняет вариацию массы (\(R^2\) близок к нулю).  
- Ни индивидуальные t-тесты, ни совместный F-тест не дают оснований считать эти признаки значимыми.

Таким образом, исходная линейная модель, собранная «как есть», не выявляет влияния характеристик экрана и батареи на массу устройства. Для адекватного описания зависимости требуется либо расширить набор переменных, либо применить нелинейные преобразования.  

## Как улучшить модель

- Вместо ширины и высоты по отдельности взять площадь экрана (sc_h × sc_w) или диагональ.
- Попробовать логарифмировать данные (например, log(mobile_wt), log(screen_area), log(battery_power)) или добавить квадрат признаков.
- Проверить, влияет ли на вес комбинация, например, sc_h × battery_power.
- Добавить другие характеристики телефона: толщину корпуса, материал, объём памяти и т. д.
- Подобрать только те переменные, которые действительно помогают объяснить вес.


## Задание 2

Во втором задании лабораторной работы рассматривается применение однофакторного дисперсионного анализа для проверки равенства средних суммарных баллов разных групп.  

Цель работы:
1. Рассчитать суммарный балл каждого ученика как сумму баллов по трём предметам: математике, чтению и письму.
2. Сгруппировать данные по фактору «раса/этническая принадлежность» (race/ethnicity).
3. Вычислить межгрупповую и внутригрупповую дисперсию суммарных баллов.
4. Построить F-статистику и оценить её значимость, чтобы проверить нулевую гипотезу  
   $$
   H_0\colon \mu_1 = \mu_2 = \dots = \mu_k,
   $$
   где $\mu_i$ — средний суммарный балл в i-й группе.
5. Сделать заключение о том, существуют ли статистически значимые различия в успеваемости между группами.

In [ ]:
df = pd.read_csv('exams_dataset.csv')
df['total_score'] = (
    df['math score'] +
    df['reading score'] +
    df['writing score']
)

factor_col = 'race/ethnicity'
levels = df[factor_col].unique()
k = len(levels)

group_data = {
    lvl: df.loc[df[factor_col] == lvl, 'total_score'].values
    for lvl in levels
}
n_i = {lvl: len(vals) for lvl, vals in group_data.items()}
N = sum(n_i.values())

grand_mean = df['total_score'].mean()
group_means = {lvl: vals.mean() for lvl, vals in group_data.items()}

SSB = sum(n_i[lvl] * (group_means[lvl] - grand_mean)**2 for lvl in levels)

SSW = sum(((vals - group_means[lvl])**2).sum() for lvl, vals in group_data.items())

df_between = k - 1
df_within  = N - k

MSB = SSB / df_between
MSW = SSW / df_within

F_stat = MSB / MSW
p_value = 1 - f.cdf(F_stat, df_between, df_within)

for lvl in levels:
    print(f"  {lvl}: n = {n_i[lvl]}, mean = {group_means[lvl]:.2f}")
print(f"\nОбщая средняя: {grand_mean:.2f}\n")
print(f"SSB = {SSB:.2f}, MSB = {MSB:.2f}")
print(f"SSW = {SSW:.2f}, MSW = {MSW:.2f}\n")
print(f"F = {F_stat:.2f}, p = {p_value:.12f}")


  group B: n = 204, mean = 195.06
  group D: n = 261, mean = 209.54
  group A: n = 77, mean = 191.66
  group C: n = 324, mean = 194.98
  group E: n = 134, mean = 223.81

Общая средняя: 202.40

SSB = 112381.64, MSB = 28095.41
SSW = 1953413.14, MSW = 1963.23

F = 14.31, p = 0.000000000023


## Результаты

- Однофакторный анализ показал статистически значимое влияние фактора **race/ethnicity** на суммарный балл (F-критерий существенно превосходит критическое значение, p < 0.0001).  
- Средние значения total_score в группах различаются: наивысшие в **group E**, наименьшие в **group A**.  

## Выводы

1. Гипотеза о равенстве средних суммарных баллов во всех этнических группах **не подтверждается**.  
2. Между отдельными парами групп существуют значимые различия в успеваемости.  